In [1]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.prompts import (
    PromptTemplate,
    ChatPromptTemplate,
    MessagesPlaceholder,
)

from langchain.chains import LLMChain


llm = ChatOpenAI(
    model_name="gpt-3.5-turbo-1106",
    temperature=0.1,
)

In [2]:
# nb_6의 memory는 """문자열""" 형태로 내역을 그대로 저장함

# 이런 식의 문자열 말고도 message 형태를 출력할 수 있음
# 이걸 """대화 기반의 메세지의 memory"""라고 함

In [3]:
template = """
    You are a helpful AI talking to a human.

    {chat_history}
    Human:{question}
    You:
"""

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history",
)

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=PromptTemplate.from_template(template),
    # template에 맞춰서 메모리를 넣기위해서
    verbose=True,
)

In [4]:
chain.predict(question="My name is Nico")
chain.predict(question="I live in Seoul")
chain.predict(question="What is my name?")



> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.

    
    Human:My name is Nico
    You:


> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.

    Human: My name is Nico
AI: Hello Nico! How can I assist you today?
    Human:I live in Seoul
    You:


> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.

    Human: My name is Nico
AI: Hello Nico! How can I assist you today?
Human: I live in Seoul
AI: That's great! Seoul is a vibrant and exciting city. Is there anything specific you need assistance with related to living in Seoul?
    Human:What is my name?
    You:


> Finished chain.


'Your name is Nico.'

In [5]:
memory.load_memory_variables({})  # 이렇게 문자열이 출력됨

{'chat_history': "Human: My name is Nico\nAI: Hello Nico! How can I assist you today?\nHuman: I live in Seoul\nAI: That's great! Seoul is a vibrant and exciting city. Is there anything specific you need assistance with related to living in Seoul?\nHuman: What is my name?\nAI: Your name is Nico."}

In [6]:
# 대화 기반 message memory 사용

In [11]:
# 문자열 기반 템플릿 대신에 ChatPromptTemplate 사용
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI talking to a human"),
        MessagesPlaceholder(variable_name="chat_history"),
        # MessagesPlaceholder는 제한 없는 양의 누가 보냈는지 알 수 없는, 예측하기 어려운 메세지를 가질 수 있음
        # 그게 시스템 메세지인지, 사람 메세지인지 혹은 AI 메세지인지 알 수 없음
        # chat_history라는 key를 가진 메모리를 가져와서 MessagesPlaceholder에 담음
        ("human", "{question}"),
        # 여기에 대화기록을 넣어야 함 -> 어떻게?? -> 대화기록에는 많은 메세지들이 쌓여있음 (시스템, 사람, AI, 사람, AI, ... 시스템, 사람, ...) => MessagesPlaceholder
    ]
)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history",
    return_messages=True,  # 문자열말고 message로 출력
)

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=prompt,
    # template에 맞춰서 메모리를 넣기위해서
    verbose=True,
)

In [12]:
chain.predict(question="My name is Nico")
chain.predict(question="I live in Seoul")
chain.predict(question="What is my name?")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a human
Human: My name is Nico

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a human
Human: My name is Nico
AI: Nice to meet you, Nico! How can I assist you today?
Human: I live in Seoul

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a human
Human: My name is Nico
AI: Nice to meet you, Nico! How can I assist you today?
Human: I live in Seoul
AI: Seoul is a vibrant and exciting city! Is there anything specific you'd like to know about Seoul or anything else you'd like to discuss?
Human: What is my name?

> Finished chain.


'Your name is Nico.'